# Gas Station Simulator Problem
#### _Edward Krueger, edkrueger@gmail.com_

## Purpose

## Overview

Alice is an owner of a small gas station that only has one pump. She is looking into adding additional pumps to her gas station and has asked you to simulate the effect.

Currently, the gas station is the only one in town; customers don't leave regardless of how long the wait is. The owner has as a deal with the a service that can provide her as much gas as she needs; we won't model the filling of the tanks.

## The simulation details

### Customer arrivals

Alice's experiences have told her a customer arrives about every 2 minutes. However, she would like to be able to change this value in the simulation to add robustness.

You should use the exponential distribution parameterized so that the mean waiting time is 2. The exponential distribution is commonly used for waiting time between events. It's a nice, simple distribution to use because it has a single parameter distribution that is directly related to its mean.

For more, see: https://en.wikipedia.org/wiki/Exponential_distribution#Applications_of_exponential_distribution.

The scipy package implements random draws from this distribution: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.expon.html.

Be careful, because scipy's parametrization is different from the one in the Wikipedia article above!


### Customer requirements

Customers, obviously, utilize pumps when they need gas. We make the assumption that a customer's gas requirement is normally distributed, centered around 15 gallons, with standard deviation 5. This roughly corresponds to 95% of customers requiring between 5 and 25 gallons of gas. Almost all customers will then take between 0 and 30 gallons of gas. If you like, verify this both numerically and mathematically. (Hint: look up the "empirical rule")

Many statistical packages only provide a standard normal distribution with mean 0 and standard deviation 1. But, by multiplying by the standard deviation and adding the mean you can transform the random variable.

Mathematically, if $z \sim N(0,1)$ and $x \sim N(\mu, \sigma)$ then $z * \sigma + \mu = x$. Note, this formula is the inverse of the formula for standardizing a normal random variable.

Scipy has the normal distribution available in the same module as the exponential distribution: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html.

To be safe, since the normal distribution has support of $(-\infty, \infty)$, you'll want to make sure that the draw is always nonnegative.

### Pump rate

Many pumps can pump at a rate faster than 10 gallons per minute, the US law mandates a cap of 10 gallons per minute. Alice's pumps are this fast.

### Customer behavior

Some customers just pump gas, but most have some lay time. They go in to buy snacks, pay for gas with cash, go to the bathroom, etc.

You'll model lay time with an exponential distribution with mean 5.

Customer's will spend the higher of their pump time and their lay time utilizing the pump and the leave allowing the next costumer to use the pump.

## Implementation

I would suggest using the package Simpy to write your solution, as that is the package I've written mine in. However, there are many other options in many languages. This is simple enough that you could even write this from scratch; but I wouldn't advise it. Simpy has a good, quick, 10-minute tutorial at https://simpy.readthedocs.io/en/latest/simpy_intro/index.html#intro.

Copyright © 2018 Edward Krueger

# Gas Station Simulator Solution

## Setup

In [1]:
import simpy

import numpy as np 

from scipy.stats import expon
from scipy.stats import norm


ModuleNotFoundError: No module named 'simpy'

In [ ]:
class GasStation():
    
    """
    Models a gas station as a simpy.Resource with a given number of pumps.
    
    Arguments:
    -----------
    num_pumps (int): The number of pumps at the gas station
    env (simpy.Environment): The simpy environment
    """
    
    def __init__(self, num_pumps, env):
        
        self.num_pumps = num_pumps
        
        self.env = env
        
        self.resource = simpy.Resource(self.env, capacity=num_pumps)
        
        self.minutes_utilized = 0
        self.gallons_sold = 0

In [ ]:
class Car():

    """
    Models a car.
    The car's simulation process in the simulation is the method .run().
    
    Arguments:
    -----------
    id (int): A unique id for the car
    gas_required (float): Amount of gallons a car will fill
    lay_time (float): Extra minutes that the car stays at the pump
    gas_station (GasStation): The gas station the cars are going to
    env (simpy.Environment): The simpy environment
    
    """
    
    def __init__(
        self,
        id,
        gas_required,
        lay_time,
        gas_station,
        env
    ):
        
        self.id = id
        self.gas_required = gas_required
        self.lay_time = lay_time
        self.gas_station = gas_station
        self.env = env
        
        self.wait_time = np.inf
        self.finished = False
        
        self.action = self.env.process(self.run())
        
    def run(self):
        
        queue_time = self.env.now
        
        print(f'Car {self.id} arrives at {round(queue_time, 2)} minutes')
                
        with self.gas_station.resource.request() as req:
            
            yield req
            
            pump_start_time = self.env.now
            
            print(f'Car {self.id} begins utilizing a pump at {round(pump_start_time, 2)} minutes')

            pump_time = self.gas_required / PUMP_RATE
            utilization = max(pump_time, self.lay_time)
            
            yield self.env.timeout(utilization)
            
            pump_end_time = self.env.now
            
            print(f'Car {self.id} leaves its pump at {round(pump_end_time, 2)} minutes')
            
            self.finished = True
            
            self.wait_time = pump_start_time - queue_time
            self.gas_station.minutes_utilized += utilization
            self.gas_station.gallons_sold += self.gas_required

In [ ]:
def scheduler(env):
    
    """
    Controls the arrival of cars at the gas station.
    
    Arguments:
    -----------
    env (simpy.Environment): The simpy environment
    """
    
    id = 0
    
    while True:
        
        waiting_time = expon.rvs(loc=0, scale=EXPECTED_WAIT)
        
        std_norm = norm.rvs()
        gas_required = std_norm * GAS_REQUIRED_STD + GAS_REQUIRED_MEAN
        gas_required = max([0, gas_required])
        
        lay_time = expon.rvs(loc=0, scale=EXPECTED_LAY_TIME)
        
        yield env.timeout(waiting_time)
        
        CARS.append(
            Car(
                id=id,
                gas_required=gas_required,
                lay_time=lay_time,
                gas_station=gas_station,
                env=env
            )
        )
        
        id += 1

## Input parameters and run the simulation

In [ ]:
EXPECTED_WAIT = 2 # in minutes

GAS_REQUIRED_MEAN = 15 # in gallons
GAS_REQUIRED_STD = 5 # in gallons

PUMP_RATE = 10 # in gallons per minute

EXPECTED_LAY_TIME = 5 # in minutes

sim_time = 24 * 60 # in minutes

## Wrap up the simulation and metrics

In [ ]:
env = simpy.Environment()
CARS = []
gas_station = GasStation(1, env)
env.process(scheduler(env))
env.run(until=sim_time)

In [ ]:
avg_wait_time = np.array([car.wait_time for car in CARS if car.finished]).mean()

total_utilization = gas_station.minutes_utilized
percent_utilization = total_utilization / (sim_time * gas_station.num_pumps)

total_gallons_sold = gas_station.gallons_sold

print(f'Average wait time (minutes): {avg_wait_time}')
print(f'Percent: {percent_utilization}')
print(f'Total Gallons Sold: {total_gallons_sold}')